In [13]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import pandas as pd

### Loading Data from Excel

In [46]:
# Load the data from Excel
excel_path = './Materials_data/面料库数据表.xlsx'  # Replace with actual file path
data = pd.read_excel(excel_path, sheet_name=None, header=None)

# Initialize an empty list to store each group's data
groups = []

# Iterate through the DataFrame in steps of 5 rows per group
for sheet_name, df in data.items():
    for i in range(0, len(df), 5):
        if pd.isna(df.iloc[i, 13]):
            break
        group_data = df.iloc[i:i+5]  # Get the 5 rows for the current group
        
        # Extract values for each row within the group
        bending_strength = group_data.iloc[0, 1:].values  # Skip first column which is the label
        strength = group_data.iloc[1, 1:].values
        deformation_strength = group_data.iloc[2, 1:].values
        deformation_rate = group_data.iloc[3, 1:].values
        filenames = group_data.iloc[4, 1:].values
        
        # Store the group data in a dictionary
        group = {
            "Bending Strength": bending_strength,
            "Strength": strength,
            "Deformation Strength": deformation_strength,
            "Deformation Rate": deformation_rate,
            "Filenames": filenames
        }
        groups.append(group)

# Display each group's data to verify
for idx, group in enumerate(groups, start=1):
    print(f"Group {idx}:")
    print("弯曲强度:", group["Bending Strength"])
    print("强度:", group["Strength"])
    print("变形强度:", group["Deformation Strength"])
    print("变形率:", group["Deformation Rate"])
    print("文件名:", group["Filenames"])
    print("\n")

Group 1:
弯曲强度: [0 10 20 30 40 50 60 70 80 90 100 nan '第一组']
强度: [20 20 20 20 20 20 20 20 20 20 20 nan nan]
变形强度: [20 20 20 20 20 20 20 20 20 20 20 nan nan]
变形率: [20 20 20 20 20 20 20 20 20 20 20 nan nan]
文件名: ['0.20.20.20' '10.20.20.20' '20.20.20.20' '30.20.20.20' '40.20.20.20'
 '50.20.20.20' '60.20.20.20' '70.20.20.20' '80.20.20.20' '90.20.20.20'
 '100.20.20.20' '文件名' nan]


Group 2:
弯曲强度: [0 10 20 30 40 50 60 70 80 90 100 nan '第二组']
强度: [40 40 40 40 40 40 40 40 40 40 40 nan nan]
变形强度: [20 20 20 20 20 20 20 20 20 20 20 nan nan]
变形率: [20 20 20 20 20 20 20 20 20 20 20 nan nan]
文件名: ['0.40.20.20' '10.40.20.20' '20.40.20.20' '30.40.20.20' '40.40.20.20'
 '50.40.20.20' '60.40.20.20' '70.40.20.20' '80.40.20.20' '90.40.20.20'
 '100.40.20.20' '文件名' nan]


Group 3:
弯曲强度: [0 10 20 30 40 50 60 70 80 90 100 nan '第三组']
强度: [60 60 60 60 60 60 60 60 60 60 60 nan nan]
变形强度: [20 20 20 20 20 20 20 20 20 20 20 nan nan]
变形率: [20 20 20 20 20 20 20 20 20 20 20 nan nan]
文件名: ['0.60.20.20' '10.60.20.20' '20.6

In [2]:
# Step 1: Data Preprocessing
def preprocess_image(image):
    # Resize and normalize image
    image = tf.image.resize(image, (105, 105))  # Standard size for example
    image = image / 255.0  # Normalize to [0, 1]
    return image

def load_data(image_pairs, labels):
    # Preprocess each image in pairs
    left_images = [preprocess_image(img[0]) for img in image_pairs]
    right_images = [preprocess_image(img[1]) for img in image_pairs]
    return (np.array(left_images), np.array(right_images)), np.array(labels)

# Example image pairs and labels (for demonstration, replace with actual data)
# Each pair should be a tuple of two images, and labels should be 1 for similar, 0 for dissimilar
# Assuming your_image_pairs is a list of tuples of images and your_labels is a list of 0 or 1 labels
# For example: your_image_pairs = [(img1, img2), (img3, img4), ...]
# your_labels = [1, 0, ...]

your_image_pairs = []  # Replace with actual image pairs
your_labels = []       # Replace with actual labels

In [4]:
# Step 2: Define the Siamese Network Model
def build_base_model():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (10, 10), activation='relu', input_shape=(105, 105, 3)),
        layers.MaxPooling2D(),
        layers.Conv2D(128, (7, 7), activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, (4, 4), activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(256, (4, 4), activation='relu'),
        layers.Flatten(),
        layers.Dense(4096, activation='sigmoid')
    ])
    return model

def build_siamese_model():
    input_left = layers.Input(name="left_input", shape=(105, 105, 3))
    input_right = layers.Input(name="right_input", shape=(105, 105, 3))

    base_model = build_base_model()

    output_left = base_model(input_left)
    output_right = base_model(input_right)

    # Compute L1 distance between features
    l1_distance = tf.abs(output_left - output_right)
    output = layers.Dense(1, activation='sigmoid')(l1_distance)

    siamese_model = Model(inputs=[input_left, input_right], outputs=output)
    return siamese_model

In [5]:
# Step 3: Compile the Model
siamese_model = build_siamese_model()
siamese_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Load your pairs and labels
(image_pairs, labels) = load_data(your_image_pairs, your_labels)

In [6]:
# Step 4: Train the Model
siamese_model.fit([image_pairs[0], image_pairs[1]], labels, batch_size=32, epochs=10)

ValueError: Expected input data to be non-empty.

In [7]:
# Step 5: Make Predictions
def predict_similarity(model, img1, img2):
    img1 = preprocess_image(img1)
    img2 = preprocess_image(img2)
    img1 = tf.expand_dims(img1, axis=0)
    img2 = tf.expand_dims(img2, axis=0)
    return model.predict([img1, img2])[0][0]

# Example usage for prediction
# Replace test_img1 and test_img2 with actual images you want to compare
# test_img1, test_img2 = <image1>, <image2>

# similarity_score = predict_similarity(siamese_model, test_img1, test_img2)
# print("Similarity score:", similarity_score)
